In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [100]:
resize_x = 64
resize_y = 64

In [101]:
classes = ['Apple','Pomegranate','Pear','Avocado','Mango']
features = {}
column_names = ['avgRGB','area','perimeter','no_of_edges','label']

In [102]:
for i in column_names:
    features[i] = []

In [103]:
def get_avg_rgb(img):
    feature = np.zeros((resize_x,resize_y))

    for i in range(resize_x):
        for j in range(resize_y):
            feature[i][j] = (int(img[i,j,0])+int(img[i,j,1])+int(img[i,j,2]))/3
    
    feature = round(np.mean(np.reshape(feature,(img.shape[0]*img.shape[1]))))
    
    return feature

In [104]:
def get_area_perimeter_edges(resized_img):
    gray = cv2.cvtColor(resized_img,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray,240,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    # cv2.imshow("Mask", thresh)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
#     height = img.shape[0]
#     width = img.shape[1]
#     print(height,width)
    # area = count/(width*height)
    cnts = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2]
    for c in cnts:
        if cv2.contourArea(c) > 100:
            perimeter = round(cv2.arcLength(c, True))
            area = cv2.contourArea(c)
            epsilon = 0.01*perimeter
            approx = cv2.approxPolyDP(c,epsilon,True)
#             cv2.drawContours(resized_img, [approx], -1, (0, 0, 255), 3)
#             cv2.imshow("Mask",resized_img)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
            edges = len(approx)
    return area,perimeter,edges

In [109]:
for i in classes:
    for f in os.listdir('C:/Users/LENOVO/harshith/Projects/ML/Neural net project/Train'+'/'+i):
        img = cv2.imread('C:/Users/LENOVO/harshith/Projects/ML/Neural net project/Train'+'/'+i+'/'+f)
        resized_img = cv2.resize(img,(resize_x,resize_y))
        rgb = get_avg_rgb(resized_img)
        area,perimeter,noOfEdges = get_area_perimeter_edges(resized_img)
        features['avgRGB'].append(rgb)
        features['area'].append(area)
        features['perimeter'].append(perimeter)
        features['no_of_edges'].append(noOfEdges)
        features['label'].append(i)

In [111]:
df = pd.DataFrame(features)
df.to_csv('features.csv',index=False)